In [5]:
print("All OK!!")

All OK!!


In [6]:
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv
import re, os
load_dotenv()

True

In [7]:
llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=.2)

In [8]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000029E3F647EE0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000029E42ACFC10>, root_client=<openai.OpenAI object at 0x0000029E3F647D00>, root_async_client=<openai.AsyncOpenAI object at 0x0000029E42ACFB80>, temperature=0.2, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [9]:
memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True)

C:\Users\ASIF\AppData\Local\Temp\ipykernel_8452\121037554.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [10]:
application_info={
    "name":None,
    "email":None,
    "skills":None
}

In [11]:
def extract_application_info(text):
    name_match=re.search(r"(?:my name is|i am|name)\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)", text, re.IGNORECASE)
    email_match=re.search(r"\b[\w.-]+@[\w.-]+\.\w+\b", text)
    skill_match=re.search(r"(?:skills are|i know | i can use|skills|Skills)\s+(.*)", text, re.IGNORECASE)
    
    response=[]
    
    if name_match:
        application_info["name"]=name_match.group(1).title()
        response.append("Name Saved.")
    if email_match:
        application_info["email"]=email_match.group(0)
        response.append("Email Saved.")
    if skill_match:
        application_info["skills"]=skill_match.group(1).strip()
        response.append("Skills Saved.")
    
    if not any([name_match, email_match, skill_match]):
        return "I could`nt extract any Info, Could you please provide your name, email or skills to complete application process"
    else:
        return " ".join(response) + " Let me check what else I need"

In [ ]:
def check_application_goal(_=None):
    if all(application_info.values()):
        return f"You are ready! Name : {application_info['name']}, Email : {application_info['email']}, Skills : {application_info['skills']}"
    else:
        missing=[k for k,v in application_info.items() if not v]
        return f" Still Need {','.join(missing)}. please ask the user to provide this."
    

In [13]:
#Creating custom tools

tools=[
    Tool(
        name="extract_application_info",
        func=extract_application_info,
        description="use this function to extract name,email and skills from the user`s message/text."
    ),
    Tool(
        name="check_application_goal",
        func=check_application_goal,
        description="Check if name, email and skills are provided.If not, tell the user what is missing.",
        return_direct=True # --important!
    )
]

In [14]:
system_prompt="""
You are a helpful job application assistant.
Your goal is to collect user`s name, email and skills.
use the tools provided to extract this information and check wether all required data is collected.
Once everything is collected, inform the user that the application info is complete and stop. 
"""

In [15]:
#initialize agent
agent=initialize_agent(
    tools=tools,
    llm=llm,
    memory=memory,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    agent_kwargs={"system_message":system_prompt}
)

C:\Users\ASIF\AppData\Local\Temp\ipykernel_8452\2515223582.py:2: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent=initialize_agent(


In [16]:
agent

AgentExecutor(memory=ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history'), verbose=True, tags=['chat-conversational-react-description'], agent=ConversationalChatAgent(llm_chain=LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.

In [ ]:
# Chat Loop
print("Hi! I am your job application assistant. Please tell me your Name, Email and Skills.")

while True:
    user_input=input("You: ")
    if user_input.lower() in ["exit","quit"]:
        print("Bye ! Good Luck.")
        break
    response=agent.invoke({"input":user_input})
    print("Bot: ", response["output"])
    
    #if goal achieved, stop
    if "You are ready!" in response["output"].lower():
        print("Application info completed")
        break
    

In [ ]:
while True:
    user_input=input("You: ")
    if user_input.lower() in ["exit","quit"]:
        print("Bye ! Good Luck.")
        break
    response=agent.invoke({"input":user_input})
    print("Bot: ", response["output"])
    
    #if goal achieved, stop
    if "You are ready!" in response["output"].lower():
        print("Application info completed")
        break